# Import Libs

In [ ]:
%matplotlib inline

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 29.3 MB/s eta 0:00:00


# Import and Clean Newsgroup Dataset

In [ ]:
from string import punctuation
from nltk import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups


newsgroups = fetch_20newsgroups()

stop_words = set(stopwords.words('english'))


tokenizer = RegexpTokenizer(r'\s+', gaps=True)


stemmer = PorterStemmer()

translet_tab = {ord(p): u" " for p in punctuation}


def text2token(raw_text):

  clean_text = raw_text.lower().translate(translet_tab)

  tokens = [token.strip() for token in tokenizer.tokenize(clean_text)]

  tokens = [token for token in tokens if token not in stop_words]

  stemmed_tokens = [stemmer.stem(token) for token in tokens]

  return [token for token in stemmed_tokens if len(token) > 2]

dataset = [text2token(txt) for txt in newsgroups['data']]

In [ ]:
from gensim.corpora import Dictionary

dictionary = Dictionary(documents=dataset, prune_at=None)

dictionary.filter_extremes(no_below=5,  no_above=0.3, keep_n=None)

dictionary.compactify()

bow_dataset = [dictionary.doc2bow(doc) for doc in dataset]

# Train model

In [ ]:
from gensim.models import LdaMulticore

num_topics = 15

lda1 = LdaMulticore(
    corpus=bow_dataset, num_topics=num_topics, id2word=dictionary,
    workers=4, eval_every=None, passes=10, batch=True
)

In [ ]:
lda2 = LdaMulticore(
        corpus=bow_dataset, num_topics=num_topics, id2word=dictionary,
    workers=4, eval_every=None, passes=10,
        alpha=(5.0/num_topics),
        eta=0.01,
        batch=True
)

# Analyizing LDA topics discovered

In [ ]:
import pyLDAvis.gensim_models
import pyLDAvis
import warnings

# Visualizing topics
pyLDAvis.enable_notebook()
LDAvis_prepered = pyLDAvis.gensim_models.prepare(lda2, bow_dataset, dictionary)
LDAvis_prepered

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.053260  0.129629       1        1  14.046308
5      0.069935  0.090725       2        1  10.470295
1     -0.058950 -0.072158       3        1   9.347860
0      0.031456 -0.066292       4        1   8.218854
4      0.052365  0.067680       5        1   7.498548
10     0.021665  0.020086       6        1   7.417414
3     -0.016687 -0.159040       7        1   7.358766
7      0.046186 -0.036117       8        1   7.100217
8      0.051970  0.026900       9        1   5.837363
2      0.021245 -0.022600      10        1   5.581919
6      0.016567  0.010231      11        1   4.184492
13     0.033827  0.024006      12        1   3.935444
11     0.015850 -0.081866      13        1   3.413746
12     0.021425  0.009952      14        1   2.962160
14    -0.360115  0.058863      15        1   2.626615, topic_info=         Term         Freq        Total Category  logprob  loglift
2398      max  4445.000000  4445.000000  Default  30.0000  30.0000
697       god  3133.000000  3133.000000  Default  29.0000  29.0000
5271      g9v  1118.000000  1118.000000  Default  28.0000  28.0000
5237      b8f  1074.000000  1074.000000  Default  27.0000  27.0000
936     space  1978.000000  1978.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
3893     char   125.308799   186.602434  Topic15  -5.8126   3.2413
806     build   150.314367  1006.244195  Topic15  -5.6307   1.7382
2619  section   131.113444   544.104297  Topic15  -5.7674   2.2164
2054     rule   133.175344   864.499125  Topic15  -5.7518   1.7690
795       air   127.793924   523.531721  Topic15  -5.7930   2.2293

[1473 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
16156     13  1.031363       0005
8860      10  0.943110       0111
6580      12  1.014366       0565
5140      15  0.975127        0tq
12488      6  0.026385  1280x1024
...      ...       ...        ...
2642       3  0.885529       zuma
2642       4  0.006236       zuma
2642       6  0.018708       zuma
2642      10  0.006236       zuma
2642      12  0.006236       zuma

[8165 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 6, 2, 1, 5, 11, 4, 8, 9, 3, 7, 14, 12, 13, 15])